####**Train Neural Network**

In [ ]:
class FaceTracker(tf.keras.Model):
    def __init__(self, base_model, **kwargs):
        super().__init__(**kwargs)
        self.model = base_model

    def compile(self, optimizer, class_loss, bbox_loss, **kwargs):
        super().compile(**kwargs)
        self.opt = optimizer
        self.closs = class_loss
        self.lloss = bbox_loss

    def train_step(self, batch):
        X, (y_class, y_bbox) = batch

        y_class = tf.cast(y_class, tf.float32)
        y_bbox  = tf.cast(y_bbox, tf.float32)

        with tf.GradientTape() as tape:
            pred_class, pred_bbox = self.model(X, training=True)

            class_loss = self.closs(y_class, pred_class)
            bbox_loss  = self.lloss(tf.cast(y_bbox, tf.float32), pred_bbox)

            total_loss = bbox_loss + 0.5 * class_loss

        grads = tape.gradient(total_loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))

        return {
            "loss": total_loss,
            "class_loss": class_loss,
            "bbox_loss": bbox_loss
        }

    def test_step(self, batch):
        X, (y_class, y_bbox) = batch

        y_class = tf.cast(y_class, tf.float32)
        y_bbox  = tf.cast(y_bbox, tf.float32)

        pred_class, pred_bbox = self.model(X, training=False)

        class_loss = self.closs(y_class, pred_class)
        bbox_loss  = self.lloss(tf.cast(y_bbox, tf.float32), pred_bbox)
        total_loss = bbox_loss + 0.5 * class_loss

        return {
            "loss": total_loss,
            "class_loss": class_loss,
            "bbox_loss": bbox_loss
        }

    def call(self, X, training=False):
        return self.model(X, training=training)

In [ ]:
model = FaceTracker(facetracker)
model.compile(
    optimizer=opt,
    class_loss=classloss,
    bbox_loss=regressloss,
    run_eagerly=True
)